In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [0]:
api_url = "https://data.urbanaillinois.us/resource/afbd-8beq.csv"

The source link of the data: https://data.urbanaillinois.us/Police/Urbana-Police-Arrests-Since-1988/afbd-8beq

In [0]:
df = pd.read_csv(api_url, delimiter = ',')

In [4]:
columns = ['arrestee_sex', 'arrestee_race', 'arrestee_residency_description', 'arrest_resolution']

for column in columns:
  print(column + '\n')  
  print(df[column].value_counts())

arrestee_sex

MALE      611
FEMALE    389
Name: arrestee_sex, dtype: int64
arrestee_race

BLACK                      520
WHITE                      346
HISPANIC                    69
ASIAN                       64
AMERICAN INDIAN/ALASKAN      1
Name: arrestee_race, dtype: int64
arrestee_residency_description

RESIDENT        500
NON-RESIDENT    422
UNKNOWN          70
Name: arrestee_residency_description, dtype: int64
arrest_resolution

TAKEN TO JAIL               386
PROMISE TO COMPLY           238
INDIVIDUAL BOND             208
NOTICE TO APPEAR - STATE     69
NOTICE TO APPEAR - CITY      54
NOT AVAILABLE                45
Name: arrest_resolution, dtype: int64


In [5]:
df[columns].head()

,arrestee_sex,arrestee_race,arrestee_residency_description,arrest_resolution
0,MALE,HISPANIC,RESIDENT,TAKEN TO JAIL
1,MALE,ASIAN,RESIDENT,TAKEN TO JAIL
2,MALE,ASIAN,RESIDENT,TAKEN TO JAIL
3,MALE,ASIAN,RESIDENT,TAKEN TO JAIL
4,MALE,ASIAN,RESIDENT,TAKEN TO JAIL


In [6]:
df.groupby(columns[:len(columns)-1]).arrest_resolution.value_counts()

arrestee_sex  arrestee_race            arrestee_residency_description  arrest_resolution       
FEMALE        AMERICAN INDIAN/ALASKAN  RESIDENT                        NOTICE TO APPEAR - CITY      1
              ASIAN                    NON-RESIDENT                    PROMISE TO COMPLY            7
                                                                       INDIVIDUAL BOND              1
                                                                       NOTICE TO APPEAR - STATE     1
                                                                       TAKEN TO JAIL                1
                                       RESIDENT                        PROMISE TO COMPLY            6
                                                                       INDIVIDUAL BOND              2
                                                                       NOTICE TO APPEAR - CITY      1
              BLACK                    NON-RESIDENT                    INDIVIDUAL BOND  

In [7]:
data = df[columns]
data[columns[len(columns) - 2]].fillna('UNKNOWN', inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Below is the requirerd data structure for D3.js Sankey chart. 

In [8]:
simple_sankey = {'nodes': [{'name': 'value'}], 'links': [{'source': 0, 'dest': 1, 'value': 100}]}
print(json.dumps(simple_sankey))

{"nodes": [{"name": "value"}], "links": [{"source": 0, "dest": 1, "value": 100}]}


'nodes' is a list of chosen columns.
'index' is used for looking up correct index value for source and dest in link creation.

Adding the nodes is trivial and the index dictionary seems to be okay. 
If there is NaN in the data, it should be handled with dropna or fillna.

In [9]:
nodes = columns
index = dict()

sample_sankey = {'nodes': []}

count = 0;
for node in nodes:
  for value in data[node].unique().tolist():
    sample_sankey['nodes'].append({'name': str(value)})
    index[value] = count
    count += 1
    
sample_sankey['links'] = list()

    
print(json.dumps(sample_sankey))
print(index)

{"nodes": [{"name": "MALE"}, {"name": "FEMALE"}, {"name": "HISPANIC"}, {"name": "ASIAN"}, {"name": "BLACK"}, {"name": "WHITE"}, {"name": "AMERICAN INDIAN/ALASKAN"}, {"name": "RESIDENT"}, {"name": "NON-RESIDENT"}, {"name": "UNKNOWN"}, {"name": "TAKEN TO JAIL"}, {"name": "INDIVIDUAL BOND"}, {"name": "PROMISE TO COMPLY"}, {"name": "NOTICE TO APPEAR - STATE"}, {"name": "NOT AVAILABLE"}, {"name": "NOTICE TO APPEAR - CITY"}], "links": []}
{'MALE': 0, 'FEMALE': 1, 'HISPANIC': 2, 'ASIAN': 3, 'BLACK': 4, 'WHITE': 5, 'AMERICAN INDIAN/ALASKAN': 6, 'RESIDENT': 7, 'NON-RESIDENT': 8, 'UNKNOWN': 9, 'TAKEN TO JAIL': 10, 'INDIVIDUAL BOND': 11, 'PROMISE TO COMPLY': 12, 'NOTICE TO APPEAR - STATE': 13, 'NOT AVAILABLE': 14, 'NOTICE TO APPEAR - CITY': 15}


create_links function takes in source and target column labels and correctly creates the links.

In [10]:
def create_links(data, sankey, source, target):
  for src in data[source].unique().tolist():
    src_data = data[data[source] == src]
    src_tgt_data = src_data[target].value_counts()
    for key, val in src_tgt_data.to_dict().items():
      link = {'source': index[src], 'target': index[key], 'value': val}
      sankey['links'].append(link)
      
create_links(data, sample_sankey, nodes[0], nodes[1])
create_links(data, sample_sankey, nodes[1], nodes[2])

print(json.dumps(sample_sankey))

{"nodes": [{"name": "MALE"}, {"name": "FEMALE"}, {"name": "HISPANIC"}, {"name": "ASIAN"}, {"name": "BLACK"}, {"name": "WHITE"}, {"name": "AMERICAN INDIAN/ALASKAN"}, {"name": "RESIDENT"}, {"name": "NON-RESIDENT"}, {"name": "UNKNOWN"}, {"name": "TAKEN TO JAIL"}, {"name": "INDIVIDUAL BOND"}, {"name": "PROMISE TO COMPLY"}, {"name": "NOTICE TO APPEAR - STATE"}, {"name": "NOT AVAILABLE"}, {"name": "NOTICE TO APPEAR - CITY"}], "links": [{"source": 0, "target": 4, "value": 344}, {"source": 0, "target": 5, "value": 175}, {"source": 0, "target": 2, "value": 47}, {"source": 0, "target": 3, "value": 45}, {"source": 1, "target": 4, "value": 176}, {"source": 1, "target": 5, "value": 171}, {"source": 1, "target": 2, "value": 22}, {"source": 1, "target": 3, "value": 19}, {"source": 1, "target": 6, "value": 1}, {"source": 2, "target": 7, "value": 41}, {"source": 2, "target": 8, "value": 27}, {"source": 2, "target": 9, "value": 1}, {"source": 3, "target": 7, "value": 38}, {"source": 3, "target": 8, "val

Using NLTK's ngram, the sources and targets are correctly organized for the create_links function.

In [11]:
from nltk.util import ngrams

bigram = list(ngrams(nodes, 2))
bigram

[('arrestee_sex', 'arrestee_race'),
 ('arrestee_race', 'arrestee_residency_description'),
 ('arrestee_residency_description', 'arrest_resolution')]

Now, the data is in correct format for Sankey diagram.

In [12]:
nodes = columns
index = dict()

sankey = {'nodes': []}

count = 0;
for node in nodes:
  for value in data[node].unique().tolist():
    sankey['nodes'].append({'name': str(value)})
    index[value] = count
    count += 1
    
sankey['links'] = list()

for gram in bigram:
  create_links(data, sankey, gram[0], gram[1])
  
  
print(json.dumps(sankey))

{"nodes": [{"name": "MALE"}, {"name": "FEMALE"}, {"name": "HISPANIC"}, {"name": "ASIAN"}, {"name": "BLACK"}, {"name": "WHITE"}, {"name": "AMERICAN INDIAN/ALASKAN"}, {"name": "RESIDENT"}, {"name": "NON-RESIDENT"}, {"name": "UNKNOWN"}, {"name": "TAKEN TO JAIL"}, {"name": "INDIVIDUAL BOND"}, {"name": "PROMISE TO COMPLY"}, {"name": "NOTICE TO APPEAR - STATE"}, {"name": "NOT AVAILABLE"}, {"name": "NOTICE TO APPEAR - CITY"}], "links": [{"source": 0, "target": 4, "value": 344}, {"source": 0, "target": 5, "value": 175}, {"source": 0, "target": 2, "value": 47}, {"source": 0, "target": 3, "value": 45}, {"source": 1, "target": 4, "value": 176}, {"source": 1, "target": 5, "value": 171}, {"source": 1, "target": 2, "value": 22}, {"source": 1, "target": 3, "value": 19}, {"source": 1, "target": 6, "value": 1}, {"source": 2, "target": 7, "value": 41}, {"source": 2, "target": 8, "value": 27}, {"source": 2, "target": 9, "value": 1}, {"source": 3, "target": 7, "value": 38}, {"source": 3, "target": 8, "val

In [0]:
with open('data.json', 'w') as file:
    json.dump(sankey, file)

Use Flow-o-matic: https://observablehq.com/@mbostock/flow-o-matic to save time.

JSON Online Viewer: http://jsonviewer.stack.hu/

D3.js Sankey: https://github.com/d3/d3-sankey